<a href="https://colab.research.google.com/github/andrewbowen19/mastersThesisData698/blob/main/GNN_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --q torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.0 MB/s eta 0:00:00


In [2]:
import torch_geometric
from torch_geometric.datasets import Reddit
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import torch
from torch_geometric.data import HeteroData

In [3]:
class DataCleaner:

  def __init__(self):
    pass

  @staticmethod
  def snake_case_columns(df: DataFrame) -> DataFrame:
    """Convert all columns in a pandas DataFrame to `snake_case`"""
    new_cols = [c.lower().replace(" ", "_").replace("-", "_") for c in df.columns]

    df.columns = new_cols
    return df

  @staticmethod
  def trim_whitespace(df: DataFrame) -> DataFrame:
    """Trim all leading and trailing whitespace"""
    for c in df.columns:
      if isinstance(df.dtypes[c], np.dtypes.ObjectDType):
        df[c] = df[c].str.strip()
    return df

  @staticmethod
  def date_parser(df: DataFrame, column_name: str, format: str = "%m/%d/%y") -> DataFrame:
    """Parse date-like columns in a dataframe"""
    df[column_name] = pd.to_datetime(df[column_name], format=format)
    return df

cleaner = DataCleaner()


## Datasets

The [CEDR Data Catalog](https://oriseapps.orau.gov/cedr/pdf/cedr-catalog-2021-508.pdf) is a good resource with documentation on the available datasets from the database.

- [Dataset 1](https://oriseapps.orau.gov/cedr/search_results.aspx?DataSet=MFMM98W1)
- [Dataset 2](https://oriseapps.orau.gov/cedr/search_results.aspx?DataSet=MFMM98W2)

## Reading in and Preprocessing our datasets

The datasets are available for [download here](https://oriseapps.orau.gov/cedr/search_results.aspx?DataSet=MFMM98W1). There are chemical agent, industrial hygiene, and building lists for the below sites:

- Hanford Site
- Los Alamos Natinoal Laboratory
- Savannah River Site
- Oak Ridge National Laboratory

We'll need to do some data wrangling before we put this into a [`HeteroData` dataset suitable for PyG](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.data.HeteroData.html#torch_geometric.data.HeteroData)

In [4]:
site_mapping = {
    "Hanford Site": {"code": "hanford",
                     "filename_prefix": "Hanford"},
    "Los Alamos National Laboratory": {"code": "lanl",
                     "filename_prefix": "LANL"},
    "Savannah River Site": {"code": "srs",
                     "filename_prefix": "SRS"},
    "Oak Ridge National Laboratory": {"code": "ornl",
                     "filename_prefix": "ORNL"},
}

data_types = ["Buildings", "ChemicalAgents", "IH"]

def construct_df(data_type: str = "Buildings"):
  dfs = []
  for site, info in site_mapping.items():
    prefix = info.get("filename_prefix")
    filename = f"/content/{prefix}-{data_type}.csv"

    dat = pd.read_csv(filename)
    dfs.append(dat)


  df = pd.concat(dfs)

  return df

# Read in datasets and combine across sites
buildings = construct_df("Buildings")
agents = construct_df("ChemicalAgents")
ih_data = construct_df("IH")

In [5]:
# Do some basic data cleaning/preprocessing

# Snake Case columns
buildings = cleaner.snake_case_columns(buildings)
agents = cleaner.snake_case_columns(agents)
ih_data = cleaner.snake_case_columns(ih_data)

# Trim whitespace
buildings = cleaner.trim_whitespace(buildings)
agents = cleaner.trim_whitespace(agents)
ih_data = cleaner.trim_whitespace(ih_data)

## Converting to a Graph Dataset

In our case, the nodes fo our grpah represent different types of entities. Hence, we'll need to constrct the a dataset representing a [heterogeneous graph for PyG](https://pytorch-geometric.readthedocs.io/en/latest/notes/heterogeneous.html?highlight=heterogeneous%20graph#creating-heterogeneous-graphs)

Here's a [helpful tutorial](https://colab.research.google.com/drive/1_eR7DXBF3V4EwH946dDPOxeclDBeKNMD?usp=sharing#scrollTo=ljgXqQRsfqNs) on converting tabular datasets to a heterogeneous graph dataste

In [6]:
# Lookup the building ID number

buildings['building_id'] = buildings.index.values

ih_data = ih_data.merge(buildings[['location', 'building_id']], on="location", how="inner")
ih_data['chemical_id'] = ih_data.index.values
ih_data['quantity'] = ih_data['quantity'].replace('', np.nan).astype(float)

# Map the site feature to integer values
site_mapping = {"LANL": 1, "SRP": 2, "ORNL": 3, "HANF": 4}
buildings['site'] = buildings['facility'].map(site_mapping)

buildings[['building_id', "site"]]

,building_id,site
0,0,4
1,1,4
2,2,4
3,3,4
4,4,4
...,...,...
791,791,3
792,792,3
793,793,3
794,794,3


In [33]:
# Set up feature tensores for both buildings and chemicals
building_features = torch.tensor(buildings[['building_id', "site"]].values, dtype=torch.long)
chemical_features = torch.tensor(ih_data[['chemical_id', 'quantity']].values, dtype=torch.long)

# Create a HeteroData object
data = HeteroData()

# Add node features (buildings and chemicals) to the HeteroData object
data['building'].x = building_features
data['chemical'].x = chemical_features

# Flip the order for the edge index so it's building (contains) chemical
edge_index = ih_data[['chemical_id', "building_id"]].values.transpose()
data['chemical', 'contained_in', 'building'].edge_index = torch.tensor(edge_index, dtype=torch.long)

In [34]:
print(f"edge_index: {edge_index.ndim}, edge_index.shape: {edge_index.shape}")
print(f"building_features: {building_features.ndim}, building_features.shape: {building_features.shape}")
print(f"chemical_features: {chemical_features.ndim}, chemical_features.shape: {chemical_features.shape}")

edge_index: 2, edge_index.shape: (2, 2335)
building_features: 2, building_features.shape: torch.Size([1562, 2])
chemical_features: 2, chemical_features.shape: torch.Size([2335, 2])


In [ ]:
# Visualizing our graph dataset
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx

# Convert HeteroData to NetworkX graph
graph = to_networkx(data)

# Visualize the NetworkX graph
plt.figure(figsize=(15, 12))

# Draw nodes for each type with different colors
node_colors = {'building': ('blue',  5),
               'chemical': ('green', 3)}
for node_type, node_setting in node_colors.items():
    nodes = [node for node, data in graph.nodes(data=True)]
    nx.draw_networkx_nodes(graph, pos=nx.spring_layout(graph),
                       nodelist=nodes, node_color=node_setting[0],
                       node_size=node_setting[1], label=None)

# Draw edges representing relationship between buildings and chemicals
edges = data['chemical', 'contained_in', 'building'].edge_index.t().tolist()
graph.add_edges_from(edges, type=('chemical', 'contained_in', 'building'))
nx.draw_networkx_edges(graph, pos=nx.spring_layout(graph), edge_color='black', arrows=False, width=0.5)

# Add legend
node_legend = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color[0], markersize=3, label=label) for label, color in node_colors.items()]
plt.legend(handles=node_legend)

plt.title("Department of Energy Chemical Agents and Buildings: 1998")
plt.axis('off')
plt.show()


## Model Building


TODO: Parse this code from the [PyG tutorial](https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html)

In [35]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, HeteroConv, Linear, SAGEConv


class HeteroGNN(torch.nn.Module):
    """
    Heterogenous Model:
      - https://pytorch-geometric.readthedocs.io/en/latest/tutorial/heterogeneous.html#using-the-heterogeneous-convolution-wrapper
    """
    def __init__(self, hidden_channels, out_channels, num_layers):
        super().__init__()

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HeteroConv({
                ('chemical', 'contained_in', 'building'): SAGEConv((-1, -1), hidden_channels)
            }, aggr='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
            x_dict = {key: x.relu() for key, x in x_dict.items()}
        return self.lin(x_dict['building'])

model = HeteroGNN(hidden_channels=64, out_channels=1,
                  num_layers=2)

/usr/local/lib/python3.10/dist-packages/torch_geometric/nn/conv/hetero_conv.py:76: UserWarning: There exist node types ({'chemical'}) whose representations do not get updated during message passing as they do not occur as destination type in any edge type. This may lead to unexpected behavior.
  warnings.warn(


In [36]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)
    loss = F.cross_entropy(out['building'], data['chemical'].y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)

train()

RuntimeError: expected m1 and m2 to have the same dtype, but got: long int != float

In [38]:
 data.validate()

True